In [1]:
import sys,os

sys.path.append('/home/arash/ProjectVR/')

import importlib
import deepsurv
import json
import time
import numpy as np
import pandas as pd
import pickle
import theano.tensor
import lasagne
import optunity

#import logging
#from logging import handlers
#from sklearn.model_selection import train_test_split

importlib.reload(deepsurv)

from deepsurv import deep_surv, utils

#from deepsurv.deepsurv_logger import DeepSurvLogger, TensorboardLogger

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

### Load trained network and data

In [2]:
#deepsurv formatted data    
with open('/home/arash/ProjectVR/cleaneddata/deepwaitdata/xtest', 'rb') as f:
    xtest=pickle.load(f)

with open('/home/arash/ProjectVR/cleaneddata/deepwaitdata/xtrain', 'rb') as f:
    xtrain=pickle.load(f)
    
#dataframe data with wait times and col names    
with open('/home/arash/ProjectVR/cleaneddata/deepwaitdata/nctest', 'rb') as f:   
    NCtest=pickle.load(f)
    
with open('/home/arash/ProjectVR/cleaneddata/deepwaitdata/nctrain', 'rb') as f:   
    NCtrain=pickle.load(f) 
    
network=deep_surv.load_model_from_json(model_fp = '/home/arash/ProjectVR/models/Deepwait100epochsTuned',
                     weights_fp = '/home/arash/ProjectVR/models/deepwait_weights_100_epochs.npz')

Loading json model: {"n_in": 19, "learning_rate": 0.0008208386784744157, "hidden_layers_sizes": [90, 90], "lr_decay": 0.000551240234375, "activation": "rectify", "dropout": 0.0, "batch_norm": true, "standardize": false}


### Predicting event times
To predict the time of event for instances, having an estimation of partial hazard is not sufficient. First, baseline hazard is required to estimate hazard function, which will be a function of time and covariates. The next step would be to calculate survival function. Finally, expected wait time can be calculated by:
\begin{equation}
    E(t)=\int_{t=0}^\infty S(t)dt
\end{equation}
### Baseline Hazard
Baseline hazard is interpreted as hazard function regardless of the values of the covariates (or when they are all 0). Different methods have been suggested in the literature for an estimation of baseline hazard. For CPH, the method used in python package is extracted from Breslow [1972]. In their method, "the underlying survival distribution is
parameterized as a continuous one, having constant hazards $\lambda_i$ between each
pair $t_{i-1}$ to $t_i$ of distinct relapse times. If all withdrawals, or censored observations, which
occur in the interval $(t_i, t_{i+1})$ are adjusted to have occurred at $t_i$ , the ML estimator of
$\lambda$ in terms of $\beta$ coefficients at time $t_i$ by:
\begin{equation}
\label{eq:9}
    \lambda_i=\frac{m_i}{(t_i-t_{i-1})*\displaystyle \sum_{j: T_j>T_i } e^{\beta*Z_j}}
\end{equation}
in which $m_i$ is the number of events at time $t_i$, Equation above is provided for CPH. In deepwait, we replace the partial hazards part with the exp of the output of our network 

deepwait predicts wait time for autonmous vehicle, so the data used will be solely those ones that the car can observe, not sociodemographic etc, and the baseline hazards are calculated based on training data, as the car will not know about other pedestrians at risk etc.

In [4]:
hazards = network.hazard(xtrain[:,0:network.hyperparams['n_in']])               #calculate partial hazard 

NCtrain['Partial Hazards'] = hazards

WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.
WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.


In [43]:
eventtimes = NCtrain.sort_values(by=['Wait Time (s)'])['Wait Time (s)'].unique() #sorted event times

In [44]:
eventnum = NCtrain['Wait Time (s)'].value_counts().sort_index() 
#m_i in the baseline hazard formula, sorted by time

In [81]:
basehazard=[]                      #Baseline hazard according to Breslow Formulation
for i in range(len(eventtimes)):
    basehazard.append(
        list(eventnum)[i] / sum(NCtrain['Partial Hazards'][NCtrain['Wait Time (s)'] >= eventtimes[i]]))

In [87]:
basehazard[100]*

0.00022284056800902033